# Abigeato, Amenazas & Delitos Sexuales

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
PATH_FILE = '/content/drive/MyDrive/Police-Data/Bronce'
OUTPUT_PATH_FILE = '/content/drive/MyDrive/Police-Data/Silver'

## Libraries

In [3]:
import pandas as pd
import numpy as np
import os
import glob

## Read Files

In [5]:
def find_header_row(file_path):
    """
    Finds the header line with a column name
    """
    column_name = ['DEPARTAMENTO','DEPARTAMENTO ','Departamento','departamento']
    header_row = None

    df = pd.read_excel(file_path, header=None)

    for i, row in df.iterrows():
        if column_name[0] in row.values or column_name[1] in row.values or column_name[2] in row.values or column_name[3] in row.values:
            header_row = i
            break
        if i>20:
            break

    if header_row is None:
        raise ValueError(f"No se encontró la fila de encabezado en el archivo {file_path}")

    return header_row

def read_excel_with_variable_header(file_path):
    header_row = find_header_row(file_path)

    df = pd.read_excel(file_path, header=header_row)

    total_row_index = df[df.iloc[:, 0].str.contains("TOTAL", case=False, na=False)].index.min()

    if total_row_index is not None:
        df = df.iloc[:total_row_index]

    return df

In [6]:
folders = ["Abigeato","Amenazas","Delitos_Sexuales"]

dataframes_by_folder = {}

def folder_read(folders):
  for folder in folders:
      file_paths = glob.glob(os.path.join(PATH_FILE+f'/{folder}/','*.xlsx')) + glob.glob(os.path.join(PATH_FILE+f'/{folder}/','*.xls'))
      folder_dataframes = []
      for file_path in file_paths:
          try:
              df = read_excel_with_variable_header(file_path)
              folder_dataframes.append(df)
          except ValueError as e:
              print(e)
      dataframes_by_folder[folder] = folder_dataframes

In [7]:
folder_read(folders)

## Abigeato

In [8]:
desired_columns = ['FECHA', 'DEPARTAMENTO', 'MUNICIPIO', 'CODIGO_DANE', 'AGRUPA_EDAD_PERSONA', 'GENERO', 'ARMAS_MEDIOS', 'CANTIDAD']
def standardize_columns(df):
    column_mapping = {
        'FECHA ': 'FECHA', 'FECHA HECHO': 'FECHA', 'FECHA HECHO ': 'FECHA',
        'DEPARTAMENTO ': 'DEPARTAMENTO',
        'MUNICIPIO ': 'MUNICIPIO',
        'CODIGO DANE': 'CODIGO_DANE', 'CODIGO DANE ': 'CODIGO_DANE',
        '*AGRUPA_EDAD_PERSONA': 'AGRUPA_EDAD_PERSONA', '*AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA', 'AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA',
        'GENERO ': 'GENERO',
        'ARMAS_MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMA MEDIO ': 'ARMAS_MEDIOS',
        'CANTIDAD ': 'CANTIDAD'
    }

    df = df.rename(columns=column_mapping)
    df = df.reindex(columns=desired_columns)
    return df

In [52]:
standardized_dataframes = [standardize_columns(df) for df in dataframes_by_folder['Abigeato']]
abigeato_df = pd.concat(standardized_dataframes, ignore_index=True)

In [53]:
agrupa_edad_persona_valid_values = ["ADULTOS","MENORES","ADOLESCENTES"]
genero_valid_values = ["MASCULINO","FEMENINO"]
armas_medios_no_valid_values = ["-"]
abigeato_df["AGRUPA_EDAD_PERSONA"] = abigeato_df["AGRUPA_EDAD_PERSONA"].apply(lambda x: x if x in agrupa_edad_persona_valid_values else "NO REPORTADO")
abigeato_df["GENERO"] = abigeato_df["GENERO"].apply(lambda x: x if x in genero_valid_values else "NO REPORTADO")
abigeato_df["ARMAS_MEDIOS"] = abigeato_df["ARMAS_MEDIOS"].apply(lambda x: x if x not in armas_medios_no_valid_values else "NO REPORTADO")

In [81]:
def clean_dane_code(data):
  temp = data.copy()
  temp['CODIGO_DANE'] = temp['CODIGO_DANE'].astype(str)
  filtered_df = temp[temp['CODIGO_DANE'].apply(lambda x: x[-1].isdigit() if x else False)]
  filtered_df.loc[:, 'CODIGO_DANE'] = filtered_df['CODIGO_DANE'].astype(float)
  return filtered_df

In [54]:
abigeato_df['CODIGO_DANE'] = abigeato_df['CODIGO_DANE'].astype(str)
filtered_df = abigeato_df[abigeato_df['CODIGO_DANE'].apply(lambda x: x[-1].isdigit() if x else False)]
filtered_df['CODIGO_DANE'] = filtered_df['CODIGO_DANE'].astype(float)

In [55]:
abigeato_output = filtered_df.groupby(["FECHA","DEPARTAMENTO","MUNICIPIO","CODIGO_DANE","AGRUPA_EDAD_PERSONA","GENERO","ARMAS_MEDIOS"]).sum().reset_index()

In [58]:
abigeato_output.to_csv(OUTPUT_PATH_FILE+"/abigeato.csv",index=False)

## Amenazas

In [59]:
desired_columns = ['FECHA', 'DEPARTAMENTO', 'MUNICIPIO', 'CODIGO_DANE', 'AGRUPA_EDAD_PERSONA', 'GENERO', 'ARMAS_MEDIOS', 'CANTIDAD']
def standardize_columns(df):
    column_mapping = {
        'FECHA ': 'FECHA', 'FECHA HECHO': 'FECHA', 'FECHA HECHO ': 'FECHA',
        'DEPARTAMENTO ': 'DEPARTAMENTO',
        'MUNICIPIO ': 'MUNICIPIO',
        'CODIGO DANE': 'CODIGO_DANE', 'CODIGO DANE ': 'CODIGO_DANE',
        '*AGRUPA_EDAD_PERSONA': 'AGRUPA_EDAD_PERSONA', '*AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA', 'AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA',
        'GENERO ': 'GENERO',
        'ARMAS_MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMA MEDIO ': 'ARMAS_MEDIOS',
        'CANTIDAD ': 'CANTIDAD'
    }

    df = df.rename(columns=column_mapping)
    df = df.reindex(columns=desired_columns)
    return df

In [60]:
standardized_dataframes = [standardize_columns(df) for df in dataframes_by_folder['Amenazas']]
amenazas_df = pd.concat(standardized_dataframes, ignore_index=True)

In [61]:
agrupa_edad_persona_valid_values = ["ADULTOS","MENORES","ADOLESCENTES"]
genero_valid_values = ["MASCULINO","FEMENINO"]
armas_medios_valid_values = ['ARMA BLANCA / CORTOPUNZANTE','ARMA DE FUEGO','CONTUNDENTES','GRANADA DE MANO',
                             'NO REPORTADO','SIN EMPLEO DE ARMAS','ESCOPOLAMINA']
amenazas_df["AGRUPA_EDAD_PERSONA"] = amenazas_df["AGRUPA_EDAD_PERSONA"].apply(lambda x: x if x in agrupa_edad_persona_valid_values else "NO REPORTADO")
amenazas_df["GENERO"] = amenazas_df["GENERO"].apply(lambda x: x if x in genero_valid_values else "NO REPORTADO")
amenazas_df["ARMAS_MEDIOS"] = amenazas_df["ARMAS_MEDIOS"].apply(lambda x: x if x in armas_medios_valid_values else "NO REPORTADO")

In [64]:
amenazas_filtered_df = clean_dane_code(amenazas_df)

In [69]:
amenazas_filtered_df['FECHA_TEMP'] = pd.to_datetime(amenazas_filtered_df['FECHA'])
amenazas_filtered_df['FECHA'] = amenazas_filtered_df['FECHA_TEMP'].dt.date
amenazas_filtered_df.drop(columns=['FECHA_TEMP'], inplace=True)

In [70]:
amenazas_output = amenazas_filtered_df.groupby(["FECHA","DEPARTAMENTO","MUNICIPIO","CODIGO_DANE","AGRUPA_EDAD_PERSONA","GENERO","ARMAS_MEDIOS"]).sum().reset_index()

In [71]:
amenazas_output.to_csv(OUTPUT_PATH_FILE+"/amenazas.csv",index=False)

## Delitos Sexuales

In [74]:
dataframes_by_folder['Delitos_Sexuales'][10]['DEPARTAMENTO_TEMP']=dataframes_by_folder['Delitos_Sexuales'][10]['ARMAS_MEDIOS']
dataframes_by_folder['Delitos_Sexuales'][10]['ARMAS_MEDIOS']=dataframes_by_folder['Delitos_Sexuales'][10]['DEPARTAMENTO']
dataframes_by_folder['Delitos_Sexuales'][10]['DEPARTAMENTO']=dataframes_by_folder['Delitos_Sexuales'][10]['DEPARTAMENTO_TEMP']
dataframes_by_folder['Delitos_Sexuales'][10].drop(columns='DEPARTAMENTO_TEMP',inplace=True)

In [76]:
desired_columns = ['FECHA', 'DEPARTAMENTO', 'MUNICIPIO', 'CODIGO_DANE', 'AGRUPA_EDAD_PERSONA', 'GENERO', 'ARMAS_MEDIOS', 'DELITO', 'CANTIDAD']
def standardize_columns(df):
    column_mapping = {
        'FECHA ': 'FECHA', 'FECHA HECHO': 'FECHA', 'FECHA HECHO ': 'FECHA',
        'DEPARTAMENTO ': 'DEPARTAMENTO',
        'MUNICIPIO ': 'MUNICIPIO',
        'CODIGO DANE': 'CODIGO_DANE', 'CODIGO DANE ': 'CODIGO_DANE',
        '*AGRUPA_EDAD_PERSONA': 'AGRUPA_EDAD_PERSONA', '*AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA', 'AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA',
        'GENERO ': 'GENERO',
        'ARMAS_MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMA MEDIO ': 'ARMAS_MEDIOS',
        'DELITO ': 'DELITO',
        'CANTIDAD ': 'CANTIDAD'
    }

    df = df.rename(columns=column_mapping)
    df = df.reindex(columns=desired_columns)
    return df

In [77]:
standardized_dataframes = [standardize_columns(df) for df in dataframes_by_folder['Delitos_Sexuales']]
delitos_sexuales_df = pd.concat(standardized_dataframes, ignore_index=True)

In [78]:
agrupa_edad_persona_valid_values = ["ADULTOS","MENORES","ADOLESCENTES"]
genero_valid_values = ["MASCULINO","FEMENINO"]
armas_medios_valid_values = ['ARMA BLANCA / CORTOPUNZANTE','CONTUNDENTES','ESCOPOLAMINA','LICOR ADULTERADO','NO REPORTADO',
                             'SIN EMPLEO DE ARMAS','ARMA DE FUEGO','CINTAS/CINTURON','CORTANTES','CORTOPUNZANTES','ESPOSAS','ARMAS BLANCAS']
delitos_sexuales_df["AGRUPA_EDAD_PERSONA"] = delitos_sexuales_df["AGRUPA_EDAD_PERSONA"].apply(lambda x: x if x in agrupa_edad_persona_valid_values else "NO REPORTADO")
delitos_sexuales_df["GENERO"] = delitos_sexuales_df["GENERO"].apply(lambda x: x if x in genero_valid_values else "NO REPORTADO")
delitos_sexuales_df["ARMAS_MEDIOS"] = delitos_sexuales_df["ARMAS_MEDIOS"].apply(lambda x: x if x in armas_medios_valid_values else "NO REPORTADO")
delitos_sexuales_df["DELITO"] = delitos_sexuales_df["DELITO"].fillna("NO REPORTADO")

In [82]:
delitos_sexuales_filtered_df = clean_dane_code(delitos_sexuales_df)

In [84]:
delitos_sexuales_output = delitos_sexuales_filtered_df.groupby(["FECHA","DEPARTAMENTO","MUNICIPIO","CODIGO_DANE","AGRUPA_EDAD_PERSONA","GENERO","ARMAS_MEDIOS","DELITO"]).sum().reset_index()

In [85]:
delitos_sexuales_output.to_csv(OUTPUT_PATH_FILE+"/delitos_sexuales.csv",index=False)